In [1]:
!pip install pyodbc sqlalchemy pandas

Defaulting to user installation because normal site-packages is not writeable


The system cannot find the path specified.


In [2]:
import pandas as pd
# import pyodbc
import sqlalchemy
# import urllib
df = pd.read_csv('Transporte_Publico_Espana.csv',na_values='not_available')

Transformation (use a different file?):
Convert data into a suitable format, normalize or scale variable and feature engineering if necessary.

# Limpieza:

In [3]:
df = df.drop_duplicates(subset='ID_Viaje', keep='first')
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Ruta'] = df['Ruta'].astype(str)
df['Tipo_Transporte'] = df['Tipo_Transporte'].astype(str)
df['Region'] = df['Region'].astype(str)
df['Dia_Semana'] = df['Dia_Semana'].astype(str)
df.dtypes
df.dropna(inplace=True)

# Subir a SQL:

In [4]:
connection_url = sqlalchemy.engine.URL.create(
    "mssql+pyodbc",
    username="ASUSSTAR\Fran",
    host="ASUSSTAR\SQLEXPRESS",
    database="master",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes",
        "authentication": "ActiveDirectoryIntegrated",
    },
)
print(connection_url)

mssql+pyodbc://ASUSSTAR%2FFran@localhost/ASUSSTAR//SQLEXPRESS/master?TrustServerCertificate=yes&authentication=ActiveDirectoryIntegrated&driver=ODBC+Driver+17+for+SQL+Server


In [5]:
engine = sqlalchemy.create_engine(connection_url)

In [6]:
df.to_sql('TransportePublicoEspana', engine, index=False, if_exists='replace')

OperationalError: (pyodbc.OperationalError) ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [67].  (67) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (67)')
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
sql_df = pd.read_sql('TransportePublicoEspana', engine)
sql_df.head()

In [ ]:
df.to_excel('TransportePublicoEspana_limpio.xlsx', index=False)

# Análisis:

Datos no realistas:
Malaga - Sevilla en coche tarda 130-140 minutos segun Google Maps. Los datos muestran que el trayecto en bus se ha completdado en rangos de tiempo muy distantes entre ellos, ademas hay registros de 60 o 20 minutos, lo que no es posible. Ademas, un viaje de duracion 23 minutus y retraso 80 minutos, no tiene sentido. Teniendo en cuenta esto voy a intentar ver que tipo de transporte es el mas eficiente basandome en la duracion, retraso, y mayor capacidad de pasajeros.

In [ ]:
df[(df['Ruta']=='Malaga - Sevilla') & (df['Tipo_Transporte']=='Autobús')].sort_values(['Fecha','Tipo_Transporte'],ascending=True).head(30)

In [ ]:
df_filtered = df[['Fecha','Numero_Pasajeros','Duracion_Viaje_Minutos','Retraso_Minutos','Tipo_Transporte','Ruta']]
df_filtered.groupby(['Ruta','Tipo_Transporte']).mean().sort_values(['Ruta','Duracion_Viaje_Minutos'],ascending=True)

# Tableau Dashboard:
https://public.tableau.com/views/TransportePublicoEspana/Dashboard1?:language=en-US&publish=yes&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link